In [1]:
# TODO: доделать как в https://stackoverflow.com/questions/43325273/sampling-from-posterior-using-custom-likelihood-in-pymc3
%matplotlib inline
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import scipy.stats as stats

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
Soft evidence имитируем монеткой (Bernoulli).

In [32]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt

with pm.Model() as model:
    failure = pm.Categorical("failure", [0.1, 0.9])
    a  = 2
    t = 1
    def logp(failure, value):
        return (failure * a - a * value).sum()
    exp_surv = pm.DensityDist('exp_surv', logp, observed={'failure':failure, 'value':t})
    step = pm.Metropolis()
    trace = pm.sample(10000, tune=5000,step=step)
    samples = trace['failure']

100%|██████████████████████████████████| 15000/15000 [00:06<00:00, 2286.82it/s]


In [93]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt
from pymc3 import Discrete


class I_am_HIV_positive(Discrete):
    def __init__(self, f_1_of_10000, *args, **kwargs):
        super(I_am_HIV_positive, self).__init__(*args, **kwargs)
        self.f_1_of_10000 = tt.as_tensor_variable(f_1_of_10000)

    def random(self, point=None, size=None):
        return tt.as_tensor_variable(1)
        
    def logp(self, value):
        n = self.f_1_of_10000
        p = 0.3

        return value-1
#         return bound(
#             binomln(n, value) + logpow(p, value) + logpow(1 - p, n - value),
#             0 <= value, value <= n,
#             0 <= p, p <= 1)        
#     def logp(self, value):
# #        f_1_of_10000 = self.f_1_of_10000
# #        if (tt.eq(f_1_of_10000, 1) and tt.eq(value, 1)):
# #           p = 0.0001
# #        elif (tt.eq(f_1_of_10000, 1) and tt.eq(value, 0)):
# #           p = 0.9999
# #        elif (tt.eq(f_1_of_10000, 0) and tt.eq(value, 1)):
# #           p = 0.5
# #        elif (tt.eq(f_1_of_10000, 0) and tt.eq(value, 0)):
# #           p = 0.5
# #        else:
# #           return tt.as_tensor_variable(-np.inf)
#        return tt.as_tensor_variable(np.log(0.5))
    
class HIV_test_false_positive(Discrete):
    def __init__(self, test_show_positive, p_I_am_HIV_positive, *args, **kwargs):
        super(HIV_test_false_positive, self).__init__(*args, **kwargs)
        self.test_show_positive = tt.as_tensor_variable(test_show_positive)
        self.p_I_am_HIV_positive = tt.as_tensor_variable(p_I_am_HIV_positive)

    def random(self, point=None, size=None):
        return tt.as_tensor_variable(1)        
        
    def logp(self, value):
        test_show_positive = self.test_show_positive
        p_I_am_HIV_positive = self.p_I_am_HIV_positive
        if(tt.eq(test_show_positive, 1)):
            if(tt.eq(p_I_am_HIV_positive,1) and tt.eq(value, 1)):
                p = 0.999
            elif(tt.eq(p_I_am_HIV_positive,1) and tt.eq(value, 0)):
                p = 0.001
            elif(tt.eq(p_I_am_HIV_positive,0) and tt.eq(value, 1)):
                p = 0.001
            elif(tt.eq(p_I_am_HIV_positive,0) and tt.eq(value, 0)):
                p = 0.999
        elif(tt.eq(test_show_positive, 0)):
            p = 0.5
        else:
            return tt.as_tensor_variable(-np.inf)
        return tt.as_tensor_variable(np.log(p))
    
with pm.Model() as model:
    f_1_of_10000 = pm.Categorical("f_1_of_10000", [0.1, 0.9])
    
    p_I_am_HIV_positive = I_am_HIV_positive('p_I_am_HIV_positive', f_1_of_10000, testval=1)
       
#    test_show_positive = pm.Categorical("test_show_positive", [0.1, 0.9])

  #  p_HIV_test_false_positive = HIV_test_false_positive('p_HIV_test_false_positive', test_show_positive, p_I_am_HIV_positive, testval=1, observed=1)
    
    step = pm.Metropolis()
    trace = pm.sample(10000, tune=5000,step=step)
    samples = trace['p_I_am_HIV_positive']
    print(sum(samples)/10000)

100%|██████████████████████████████████| 15000/15000 [00:09<00:00, 1578.92it/s]


269738.2175
